In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import glob
import os
import re
import csv

import tensorflow as tf
import torch


from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

!pip install transformers
!pip install xformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline


In [ ]:
def addBertCheckpoint(inputFilePath, outputFile="lyrics_bert.csv", checkpointFile="bert_checkpoint.txt", batchSize=100):
    # Initialize the checkpoint
    if os.path.exists(checkpointFile):
        with open(checkpointFile, 'r') as f:
            checkpoint = f.read()
            startRow = int(checkpoint or 0)
    else:
        startRow = 0

    df = pd.read_csv(inputFilePath)

    # If output file exists, read it and get the 'bert' column. Otherwise, initialize 'bert' column with None.
    if os.path.exists(outputFile):
        df_out = pd.read_csv(outputFile)
        df['bert'] = df_out['bert']
    else:
        df['bert'] = [None]*len(df)

    nlp = pipeline('sentiment-analysis', model='distilbert-base-uncased', max_length=512, truncation=True)

    # Process the rows in batches, starting from the last checkpoint
    for i in tqdm(range(startRow, len(df), batchSize)):
        endRow = min(i + batchSize, len(df))
        for j in range(i, endRow):
            if pd.isna(df.loc[j, 'bert']):
                result = nlp(str(df.loc[j]))[0]
                df.loc[j, 'bert'] = -result['score'] if result['label'] == 'NEGATIVE' else result['score']
        df.to_csv(outputFile, index=False)


        # Update the checkpoint
        with open(checkpointFile, 'w') as f:
            f.write(str(endRow))

In [ ]:
#tqdm.pandas()
#df = pd.read_csv("testCollab.csv")
#addBertCheckpoint("testCollab.csv")



#addBertCheckpoint("trends_rising.csv")



In [ ]:
trends_df = pd.read_csv('testCollab.csv')
trends_df.columns = ["Rising", "Weight"]
#trends_df = trends_df.transpose()

In [ ]:
print(trends_df)

trends_rising = []
trends_weight = []

for i in trends_df["Rising"]:
  i = i.replace("[", "")
  i = i.replace("]", "")
  i = i.replace("'", "")
  i = i.replace('"', "")
  #i = i.strip()
  print(i)

  txt = i.split(", ")
  print(txt)
  trends_rising.append(txt)


for i in trends_df["Weight"]:
  i = i.replace("[", "")
  i = i.replace("]", "")
  i = i.replace("'", "")
  i = i.replace('"', "")
  #i = i.strip()
  print(i)

  txt = i.split(", ")
  print(txt)
  trends_weight.append(txt)

  #for c in i:
    #print(c)

#trends_df.head

                                                Rising  \
0    ['"alexis ohanian"', '"ronda rousey fight"', '...   
1    ['"golden globes"', '"meryl streep"', '"golden...   
2    ['"eddie long"', '"john lewis"', '"bishop eddi...   
3    ['"yordano ventura"', '"barron trump"', '"trum...   
4    ['"mary tyler moore"', '"john hurt"', '"sally ...   
..                                                 ...   
257  ['"georges seurat"', '"ethan crumbley"', '"bob...   
258  ['"demaryius thomas"', '"tornado kentucky"', '...   
259  ['"émilie du châtelet"', '"winter season"', '"...   
260  ['"norad santa tracker"', '"santa tracker"', '...   
261  ['"betty white"', '"antonio brown"', '"did bet...   

                                                Weight  
0    ['"+3,050%"', '"+2,350%"', '"+2,000%"', '"+1,7...  
1    ['"+2,850%"', '"+1,900%"', '"+1,850%"', '"+1,7...  
2    ['"+1,600%"', '"+1,400%"', '"+1,050%"', '"+800...  
3    ['"+2,050%"', '"+2,050%"', '"+1,650%"', '"+1,5...  
4    ['"Breakout"'

In [ ]:
#print(trends_rising)

for i in trends_rising:
  print(i)

+1,600%, +1,300%, +1,000%, +650%, +600%, +550%, +500%, +500%, +450%, +400%, +350%, +300%, +300%, +180%, +170%, +130%, +100%, +90%, +90%, +80%, +70%, +60%, +40%
['+1,600%', '+1,300%', '+1,000%', '+650%', '+600%', '+550%', '+500%', '+500%', '+450%', '+400%', '+350%', '+300%', '+300%', '+180%', '+170%', '+130%', '+100%', '+90%', '+90%', '+80%', '+70%', '+60%', '+40%']
+1,700%, +1,400%, +1,300%, +1,250%, +750%, +700%, +700%, +500%, +500%, +450%, +350%, +350%, +350%, +350%, +300%, +300%, +250%, +200%, +200%, +190%, +180%, +170%, +160%, +150%, +140%
['+1,700%', '+1,400%', '+1,300%', '+1,250%', '+750%', '+700%', '+700%', '+500%', '+500%', '+450%', '+350%', '+350%', '+350%', '+350%', '+300%', '+300%', '+250%', '+200%', '+200%', '+190%', '+180%', '+170%', '+160%', '+150%', '+140%']
Breakout, +2,200%, +1,850%, +1,850%, +1,650%, +1,550%, +1,550%, +1,250%, +1,100%, +750%, +700%, +700%, +550%, +550%, +500%, +500%, +300%, +250%, +250%, +250%, +190%, +180%, +180%, +160%, +160%
['Breakout', '+2,200%',

In [ ]:
trends_rising = pd.DataFrame([trends_rising, trends_weight])
trends_rising = trends_rising.transpose()
print(trends_rising)
trends_rising.to_csv('trends_rising.csv', index=False)

                                                     0  \
0    [alexis ohanian, ronda rousey fight, hollyweed...   
1    [golden globes, meryl streep, golden globes 20...   
2    [eddie long, john lewis, bishop eddie long, a ...   
3    [yordano ventura, barron trump, trump inaugura...   
4    [mary tyler moore, john hurt, sally yates, mis...   
..                                                 ...   
257  [georges seurat, ethan crumbley, bob dole, geo...   
258  [demaryius thomas, tornado kentucky, vicente f...   
259  [émilie du châtelet, winter season, donald par...   
260  [norad santa tracker, santa tracker, santa tra...   
261  [betty white, antonio brown, did betty white d...   

                                                     1  
0    [+3,050%, +2,350%, +2,000%, +1,700%, +1,400%, ...  
1    [+2,850%, +1,900%, +1,850%, +1,750%, +1,750%, ...  
2    [+1,600%, +1,400%, +1,050%, +800%, +700%, +650...  
3    [+2,050%, +2,050%, +1,650%, +1,500%, +1,400%, ...  
4    [Breakout, +3

In [ ]:
#trends_rising = trends_rising.T
#print(trends_rising)

In [ ]:
#!pip install vaderSentiment
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#model_name = 'bert_base-cased'
#tokenizer = BertTokenizer.from_pretrained(model_name)

#model_name = "finiteautomata/bertweet-base-sentiment-analysis"
model_name = "assemblyai/distilbert-base-uncased-sst2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

total_bert = []

for item in trends_rising.iterrows():
  #item = pd.DataFrame(item)
  print("FULL:")
  print(item)

  #nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, max_length=512, truncation=True)
  nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

  #sia = SentimentIntensityAnalyzer()
  #item2 = item.apply(lambda x: sia.polarity_scores(x)["compound"])

    #item2 = item.apply(lambda x: nlp(x)[0]['score'], axis=1)
    #item2 = item2.apply(lambda x: -x['score'] if x['label'] == 'NEGATIVE' else x['score'])
    #print(item2)

  date = []

  for i in item[1][0]:
    print("WITHIN:")
    print(i)
    item2 = nlp(i)
    #item2 = i.apply(lambda x: nlp(x)[0]['score'], axis=1)
    #if item2:
    #item2 = item2.apply(lambda x: -x['score'] if x['label'] == 'NEGATIVE' else x['score'])
    print(item2)
    if item2[0]['label']=='LABEL_0':
      print("DECLARING NEGATIVE")
      item2[0]['label']='NEGATIVE'
      item2[0]['score']=(-1)*item2[0]['score']
    else:
      print("DECLARING POSITIVE")
      item2[0]['label']='POSITIVE'

    print(item2)
    #print(item2[0]['label'])
    date.append(item2)

  total_bert.append(date)

FULL:
(0, 0    [alexis ohanian, ronda rousey fight, hollyweed...
1    [+3,050%, +2,350%, +2,000%, +1,700%, +1,400%, ...
Name: 0, dtype: object)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Streaming output truncated to the last 5000 lines.
what lies below
[{'label': 'LABEL_0', 'score': 0.7972205877304077}]
DECLARING NEGATIVE
[{'label': 'NEGATIVE', 'score': -0.7972205877304077}]
WITHIN:
rege jean page
[{'label': 'LABEL_1', 'score': 0.5386891961097717}]
DECLARING POSITIVE
[{'label': 'POSITIVE', 'score': 0.5386891961097717}]
WITHIN:
champions league
[{'label': 'LABEL_1', 'score': 0.6099853515625}]
DECLARING POSITIVE
[{'label': 'POSITIVE', 'score': 0.6099853515625}]
WITHIN:
mlb
[{'label': 'LABEL_0', 'score': 0.6088640093803406}]
DECLARING NEGATIVE
[{'label': 'NEGATIVE', 'score': -0.6088640093803406}]
FULL:
(223, 0    [metropolitan museum of art, prince philip, th...
1    [Breakout, +4,150%, +3,650%, +3,600%, +3,250%,...
Name: 223, dtype: object)
WITHIN:
metropolitan museum of art
[{'label': 'LABEL_1', 'score': 0.8036158680915833}]
DECLARING POSITIVE
[{'label': 'POSITIVE', 'score': 0.8036158680915833}]
WITHIN:
prince philip
[{'label': 'LABEL_1', 'score': 0.7100542187690735}]


In [ ]:
print(len(total_bert))

262


In [ ]:
print(total_bert)

In [ ]:
pd.DataFrame(total_bert).to_csv('trends_bert_sa.csv')

In [15]:
total_bert = pd.DataFrame(total_bert)

In [16]:
print(total_bert)

                                                    0   \
0    [{'label': 'POSITIVE', 'score': 0.679971218109...   
1    [{'label': 'POSITIVE', 'score': 0.934052526950...   
2    [{'label': 'NEGATIVE', 'score': -0.68316525220...   
3    [{'label': 'POSITIVE', 'score': 0.670913577079...   
4    [{'label': 'NEGATIVE', 'score': -0.52952682971...   
..                                                 ...   
257  [{'label': 'POSITIVE', 'score': 0.774658381938...   
258  [{'label': 'NEGATIVE', 'score': -0.50463634729...   
259  [{'label': 'POSITIVE', 'score': 0.744445443153...   
260  [{'label': 'NEGATIVE', 'score': -0.69301658868...   
261  [{'label': 'NEGATIVE', 'score': -0.56137531995...   

                                                    1   \
0    [{'label': 'NEGATIVE', 'score': -0.74848544597...   
1    [{'label': 'NEGATIVE', 'score': -0.66797590255...   
2    [{'label': 'POSITIVE', 'score': 0.705798387527...   
3    [{'label': 'POSITIVE', 'score': 0.591957569122...   
4    [{'label

In [28]:
import numpy as np

bertMean = []
bertSum = []

for item in total_bert.iterrows():
  print("------------------------------")
  print(item[1])
  meaner = []
  summer = 0
  for subitem in item[1]:
    if (subitem!=None):
      meaner.append(subitem[0]['score'])
      print(subitem[0])
      print(meaner)
      summer += subitem[0]['score']
      print(summer)

  mean = np.mean(meaner)
  print(mean)
  print(summer)
  bertMean.append(mean)
  bertSum.append(summer)

Streaming output truncated to the last 5000 lines.
{'label': 'NEGATIVE', 'score': -0.7704794406890869}
[0.6366223096847534, -0.5600718855857849, -0.5057488083839417, -0.5100070834159851, -0.5827835202217102, -0.5624992251396179, -0.5389338731765747, -0.678968608379364, 0.7680156826972961, -0.7854612469673157, -0.5310561656951904, 0.5144339799880981, -0.9427151083946228, -0.6687436699867249, -0.6841577887535095, -0.562385618686676, -0.608212411403656, -0.796245813369751, -0.5606792569160461, -0.7847917675971985, -0.7924350500106812, -0.6377846598625183, -0.7704794406890869]
-11.145089030265808
{'label': 'NEGATIVE', 'score': -0.9076744914054871}
[0.6366223096847534, -0.5600718855857849, -0.5057488083839417, -0.5100070834159851, -0.5827835202217102, -0.5624992251396179, -0.5389338731765747, -0.678968608379364, 0.7680156826972961, -0.7854612469673157, -0.5310561656951904, 0.5144339799880981, -0.9427151083946228, -0.6687436699867249, -0.6841577887535095, -0.562385618686676, -0.6082124114036

In [29]:
print(bertMean)

[-0.05006727933883667, -0.17086998462677003, -0.5214034724235534, 0.11096070528030395, -0.31978015422821043, -0.1637296986579895, 0.0974589991569519, -0.21731577157974244, 0.14239710569381714, -0.3586641748746236, -0.5728264689445496, -0.21813628911972047, -0.4126193415034901, -0.6269131016731262, -0.1668606948852539, -0.18929198026657104, -0.06781095504760742, -0.3385423844059308, -0.24549774408340455, 0.11903339862823487, -0.20502591133117676, -0.17911777198314666, -0.17356474399566652, -0.18016408681869506, -0.15912630558013915, -0.1934065378230551, 0.13308919906616212, -0.308276515007019, -0.18144115447998047, -0.01155961275100708, -0.26993058976672946, -0.29851533638106453, -0.17675445556640626, -0.4145050144195557, -0.4514291524887085, -0.4995137619972229, -0.28422487258911133, -0.2838120311498642, -0.14418994188308715, -0.2682722878456116, -0.5454632139205933, -0.18163953542709352, -0.44168088912963865, -0.23021192073822022, -0.3694080591201782, -0.18897443294525146, -0.02698584

In [30]:
print(len(bertMean))

262


In [31]:
print(bertSum)

[-1.2516819834709167, -4.2717496156692505, -13.035086810588837, 2.774017632007599, -7.994503855705261, -4.0932424664497375, 2.4364749789237976, -5.432894289493561, 3.5599276423454285, -8.607940196990967, -14.320661723613739, -5.4534072279930115, -4.538812756538391, -15.672827541828156, -4.171517372131348, -4.732299506664276, -1.6952738761901855, -8.12501722574234, -6.1374436020851135, 2.9758349657058716, -5.330673694610596, -3.5823554396629333, -4.339118599891663, -4.504102170467377, -3.978157639503479, -4.448350369930267, 3.3272299766540527, -7.706912875175476, -4.536028861999512, -0.288990318775177, -5.668542385101318, -5.373276054859161, -4.418861389160156, -10.362625360488892, -11.285728812217712, -12.487844049930573, -7.105621814727783, -5.676240622997284, -3.604748547077179, -6.706807196140289, -13.636580348014832, -4.540988385677338, -11.042022228240967, -5.755298018455505, -9.235201478004456, -4.724360823631287, -0.6746461987495422, -0.08222967386245728, -0.13504314422607422, -

In [32]:
print(len(bertSum))

262


In [34]:
bertTrends = pd.DataFrame([bertMean, bertSum])
bertTrends = bertTrends.T
bertTrends.columns = ['Trends Bert Mean', 'Trends Bert Sum']
print(bertTrends)

     Trends Bert Mean  Trends Bert Sum
0           -0.050067        -1.251682
1           -0.170870        -4.271750
2           -0.521403       -13.035087
3            0.110961         2.774018
4           -0.319780        -7.994504
..                ...              ...
257         -0.179629        -4.490720
258         -0.229584        -5.739590
259         -0.348724        -8.718106
260          0.120229         3.005734
261         -0.080491        -2.012285

[262 rows x 2 columns]


In [36]:
pd.DataFrame(bertTrends).to_csv('bertTrends.csv', index=False)

In [ ]:
print("BERT")
i['weighted_sentiment_bert'] = pd.to_numeric(i['bert_score'])*pd.to_numeric(i['"Weight"'])
weightedMeanBert = np.mean(i['weighted_sentiment_bert'])
print(weightedMeanBert)
weightedMeansBert.append(weightedMeanBert)
weightedSumBert = np.sum(i['weighted_sentiment_bert'])
print(weightedSumBert)
weightedSumsBert.append(weightedSumBert)

meanScoreBert = np.mean(i['bert_score'])
print(meanScoreBert)
meanBert.append(meanScoreBert)

sumScoreBert = np.sum(i['bert_score'])
print(sumScoreBert)
sumBert.append(sumScoreBert)
print()